In [5]:
import pandas as pd
import yfinance as yf
import os
from datetime import datetime, timedelta
import logging
import time

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def fetch_tickers_from_csv(file_name):
    """Fetch tickers from a CSV file located in the same directory as the script."""
    try:
        script_dir = os.getcwd()  # Get the current working directory
        file_path = os.path.join(script_dir, file_name)
        df = pd.read_csv(file_path)
        tickers = df['Ticker'].dropna().tolist()
        logging.info(f"Successfully fetched {len(tickers)} tickers from {file_name}.")
        return tickers
    except Exception as e:
        logging.error(f"Error fetching tickers from {file_name}: {e}")
        return []

def fetch_and_collect_data(ticker, start_date, end_date, retries=5):
    """Fetch stock data for a given ticker and return it as a DataFrame."""
    attempt = 0
    while attempt < retries:
        try:
            # Fetch historical data for the ticker
            data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'), progress=False)
            if not data.empty:
                data['Ticker'] = ticker  # Add a column for the ticker symbol
                data.reset_index(inplace=True)  # Ensure 'Date' column is present
                logging.info(f"Successfully fetched data for {ticker}")
                return data
            else:
                logging.warning(f"No data available for {ticker}")
                return pd.DataFrame()  # Return empty DataFrame if no data
        except Exception as e:
            logging.error(f"Error fetching data for {ticker} on attempt {attempt + 1}: {e}")
            attempt += 1
            time.sleep(5)  # Wait a bit before retrying
    logging.error(f"Failed to fetch data for {ticker} after {retries} attempts.")
    return pd.DataFrame()  # Return empty DataFrame on failure

def clean_data(df):
    """Clean the stock data by handling missing values and removing duplicates."""
    # Handle missing values
    df.ffill(inplace=True)
    df.bfill(inplace=True)

    # Remove duplicates
    df.drop_duplicates(inplace=True)

    # Ensure correct data types
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])

    numerical_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    for col in numerical_cols:
        if col in df.columns:
            df[col] = df[col].astype(float)
    
    return df

def calculate_indicators(data, threshold, ma_length=50):
    """Calculate technical indicators: Bollinger Bands, VWAP, Squeeze, etc."""
    data['MA50'] = data['Close'].rolling(window=ma_length).mean()
    data['MiddleBB'] = data['Close'].rolling(window=20).mean()
    data['UpperBB'] = data['MiddleBB'] + (data['Close'].rolling(window=20).std() * 2)
    data['LowerBB'] = data['MiddleBB'] - (data['Close'].rolling(window=20).std() * 2)
    data['BB_Width'] = (data['UpperBB'] - data['LowerBB']) / data['MiddleBB']
    data['Squeeze'] = (data['BB_Width'] < threshold).astype(int)
    data['VWAP'] = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()
    data['Low_Lower_Than_Previous'] = (data['Low'] < data['Low'].shift(1)).astype(int)
    data['Close_Above_VWAP'] = (data['Close'] > data['VWAP']).astype(int)
    data['Volume_Increase'] = (data['Volume'] > data['Volume'].rolling(window=5).mean()).astype(int)
    data['Volume_Spike'] = (data['Volume'] > data['Volume'].rolling(window=20).mean() * 1.5).astype(int)
    data['Divergence'] = ((data['Close'] - data['MA50']) / data['MA50']).abs()
    return data

def detect_spring(data, ma_col, rebound_days, rebound_percentage):
    """Identify potential Spring patterns with enhanced conditions."""
    data['Spring'] = ((data['Low_Lower_Than_Previous'] == 1) &
                      (data['Close_Above_VWAP'] == 1) &
                      (data['Volume_Increase'] == 1) &
                      (data['BB_Width'] <= 0.05) &
                      (data['Low'] < data[ma_col]) &
                      (data['Volume_Spike'] == 1)).astype(int)
    return data

def confirm_spring(data, ma_col, rebound_days, rebound_percentage):
    """Confirm Spring patterns by checking future price movements."""
    squeeze_spring = data[(data['Squeeze'] == 1) & (data['Spring'] == 1)].copy()
    squeeze_spring['Days_Between'] = (squeeze_spring['Date'].diff().dt.days.fillna(0).astype(int))  # Use 'Date' for Days_Between
    
    # Initialize 'Confirmed_Spring' with 0
    squeeze_spring['Confirmed_Spring'] = 0

    # Confirm spring by checking rebound above the moving average within the specified days
    for i in range(len(squeeze_spring)):
        date = squeeze_spring.iloc[i]['Date']
        next_days = data.loc[data['Date'] > date].head(rebound_days)  # Use 'Date' to find next days
        if len(next_days) > 1:
            for j in range(len(next_days)):  # Check each subsequent day
                if next_days.iloc[j]['Close'] > squeeze_spring.iloc[i][ma_col] * rebound_percentage:
                    squeeze_spring.at[squeeze_spring.index[i], 'Confirmed_Spring'] = 1
                    break

    return squeeze_spring[squeeze_spring['Confirmed_Spring'] == 1]

def process_stock_data(ticker, start_date, end_date, threshold, rebound_days=3, rebound_percentage=1.02):
    """Fetch, clean, analyze, and return data for stocks matching the squeeze and spring criteria."""
    data = fetch_and_collect_data(ticker, start_date, end_date)
    if data.empty:
        return None

    # Clean data
    data = clean_data(data)

    # Calculate indicators and detect patterns
    data = calculate_indicators(data, threshold)
    data = detect_spring(data, 'MA50', rebound_days, rebound_percentage)
    confirmed_springs = confirm_spring(data, 'MA50', rebound_days, rebound_percentage)
    
    # Ensure that Ticker is not None
    confirmed_springs['Ticker'] = ticker
    
    return confirmed_springs[['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 
                              'Squeeze', 'Spring', 'Days_Between', 'Confirmed_Spring']]

# Main execution
if __name__ == "__main__":
    # Define time range: Last 10 years to today
    end_date = datetime.now()
    start_date = end_date - timedelta(days=10*365)

    # Define Bollinger Band threshold
    bollinger_threshold = 0.05

    # Fetch tickers from all sources
    russell_tickers = fetch_tickers_from_csv('Russel_3000_ticker_list.csv')
    tsx_tickers = fetch_tickers_from_csv('TSX List.csv')
    tsxv_tickers = fetch_tickers_from_csv('TSXV.csv')
    cse_tickers = fetch_tickers_from_csv('CSE Listings.csv')

    # Combine all tickers into one list
    all_tickers = russell_tickers + tsx_tickers + tsxv_tickers + cse_tickers

    results = []

    # Process each ticker
    for ticker in all_tickers:
        logging.info(f"Processing {ticker}...")
        result = process_stock_data(ticker, start_date, end_date, bollinger_threshold)
        if result is not None and not result.empty:
            results.append(result)

    # Combine and save results
    if results:
        combined_results = pd.concat(results, ignore_index=True)
        combined_results.to_parquet("all_stocks_squeeze_spring_results.parquet")
        logging.info("Successfully saved all results to all_stocks_squeeze_spring_results.parquet")
    else:
        logging.info("No confirmed squeeze followed by spring patterns found.")


2024-09-01 07:59:55,746 - INFO - Successfully fetched 2634 tickers from Russel_3000_ticker_list.csv.
2024-09-01 07:59:55,748 - INFO - Successfully fetched 2484 tickers from TSX List.csv.
2024-09-01 07:59:55,750 - INFO - Successfully fetched 1515 tickers from TSXV.csv.
2024-09-01 07:59:55,752 - INFO - Successfully fetched 786 tickers from CSE Listings.csv.
2024-09-01 07:59:55,757 - INFO - Processing A...
2024-09-01 07:59:56,106 - INFO - Successfully fetched data for A
2024-09-01 07:59:56,121 - INFO - Processing AA...
2024-09-01 07:59:56,604 - INFO - Successfully fetched data for AA
2024-09-01 07:59:56,614 - INFO - Processing AAL...
2024-09-01 07:59:56,743 - INFO - Successfully fetched data for AAL
2024-09-01 07:59:56,754 - INFO - Processing AAN...
2024-09-01 07:59:57,007 - INFO - Successfully fetched data for AAN
2024-09-01 07:59:57,016 - INFO - Processing AAON...
2024-09-01 07:59:57,226 - INFO - Successfully fetched data for AAON
2024-09-01 07:59:57,236 - INFO - Processing AAP...
2024-